# 資料合併與步驟 (Python)

# 目錄

[為什麼要資料合併?](#1)  
[如何合併?](#2)  
[資料介紹](#3)  
[流程](#4)
* [觀察資料](#5)
* [資料預處理](#6)
* [資料合併](#7)


## 為什麼要資料合併?<a id = 1><a>
最常會使用資料合併的，應該是SQL查詢。在關聯式資料庫的架構下，資料平時的存放會被切割成不重複且可以藉由索引串起關聯的資料表。如果要查詢特定資料的時候，就需要使用資料合併。  

從資料探勘的角度來說，適合的變項才能有效的分類或預測，有時資料裡面會缺乏我們認為適合的變項，這時候就可以與外部資料合併，獲取需要的變項。

---

## 如何合併?<a id = 2><a>
在Python中，可以使用**Pandas**的`merge`以及`concat`進行合併資料，兩者的差異是merge的合併概念是基於關聯式資料庫，跟SQL的join很像；concat通常都是單純的把資料併列在一起，不考慮欄位之間的關聯性，不過也可以藉由參數的設定來達到類似merge的合併概念，只是比較麻煩。  
後續會用Kaggle下載的dataset來示範。

Merge跟SQL的Join非常相似，想知道語法上的邏輯差異，可以參考以下文章：  
[JOINs in SQL, Python, and R](https://mode.com/blog/joins-in-sql-python-r)  
[Comparison with SQL](https://pandas.pydata.org/pandas-docs/stable/getting_started/comparison/comparison_with_sql.html)  


這邊有直接到合併資料的[傳送門](#7)

---

## 資料介紹<a id = 3><a>
這份Kaggle Dataset的提供者應該是個登山愛好者，他提到**登頂成功與否往往會受到天氣的影響**，所以他從兩處蒐集資料了Mount Rainier National Park的**登山統計**與**天氣**的歷史資料，提供給大家，看看是否可以從這些資料能夠**預測登頂的成功率**。  

以下是兩份資料集的資料描述：

登山統計:
<img src="./圖片4.jpg" width=40% >


天氣:
<img src="./圖片5.jpg" width=40%>

資料來源: [Mount Rainier Weather and Climbing Data](https://www.kaggle.com/codersree/mount-rainier-weather-and-climbing-data), Kaggle Dataset

---

## 流程<a id = 4><a>
我會按照以下步驟來完成資料合併。

1. 載入資料
2. 觀察資料
3. 資料預處理(Data Preprocessing)
4. 資料合併 (Data Merge)

其實資料合併前，需要經過前面的步驟才能合併，即使拿到的資料已經整理過了，仍可能需要一些調整，所以觀察跟預處理也是合併之前的重要工作。  



### 載入資料
在開始之前，要先載入package與data

載入package，pandas是載入data跟合併資料都會用到的package

In [1]:
import pandas as pd

載入data，如果是Excel檔，就要用`pd.read_excel`載入

In [2]:
data1 = pd.read_csv(".\\mount-rainier-weather-and-climbing-data\\climbing_statistics.csv",engine ="python")
data2 = pd.read_csv(".\\mount-rainier-weather-and-climbing-data\\Rainier_Weather.csv",engine = "python")

### 觀察資料<a id = 5><a>
觀察資料是要確認資料的格式是否正確，有無空值等

In [3]:
data1.info() # column name 有亂碼 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4077 entries, 0 to 4076
Data columns (total 5 columns):
嚜澳ate                 4077 non-null object
Route                 4077 non-null object
Attempted             4077 non-null int64
Succeeded             4077 non-null int64
Success Percentage    4077 non-null float64
dtypes: float64(1), int64(2), object(2)
memory usage: 159.3+ KB


In [4]:
data1.head() # 日期是 mm/dd/yyyy

,嚜澳ate,Route,Attempted,Succeeded,Success Percentage
0,11/27/2015,Disappointment Cleaver,2,0,0.0
1,11/21/2015,Disappointment Cleaver,3,0,0.0
2,10/15/2015,Disappointment Cleaver,2,0,0.0
3,10/13/2015,Little Tahoma,8,0,0.0
4,10/9/2015,Disappointment Cleaver,2,0,0.0


In [5]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 464 entries, 0 to 463
Data columns (total 7 columns):
Date                     464 non-null object
Battery Voltage AVG      464 non-null float64
Temperature AVG          464 non-null float64
Relative Humidity AVG    464 non-null float64
Wind Speed Daily AVG     464 non-null float64
Wind Direction AVG       464 non-null float64
Solare Radiation AVG     464 non-null float64
dtypes: float64(6), object(1)
memory usage: 25.5+ KB


In [6]:
data2.head() #日期問題同 data1

,Date,Battery Voltage AVG,Temperature AVG,Relative Humidity AVG,Wind Speed Daily AVG,Wind Direction AVG,Solare Radiation AVG
0,12/31/2015,13.845000,19.062917,21.870833,21.977792,62.325833,84.915292
1,12/30/2015,13.822917,14.631208,18.493833,3.540542,121.505417,86.192833
2,12/29/2015,13.834583,6.614292,34.072917,0.000000,130.291667,85.100917
3,12/28/2015,13.710417,8.687042,70.557917,0.000000,164.683750,86.241250
4,12/27/2015,13.362500,14.140417,95.754167,0.000000,268.479167,31.090708


從以上可以發現
1. data1的column name 有亂碼
2. Date的日期格式是mm/dd/yyyy
3. 小數點位數太多

接下來在資料預處理的部分，調整這些問題

### 資料預處理<a id = 6><a>

**修正亂碼的column name**  
可以使用 `df.rename(colums = {original name : new name})` 來改名，如果名稱亂碼打不出來或者想用別的方法，  
也可以用 `df.columns.values[] = "new name"`，[ ]裡面填上欲修改欄位的位置

In [7]:
data1 = data1.rename(columns = {"嚜澳ate":"Date"}) # or data1.columns.values[0] = "Date"
data1.columns

Index(['Date', 'Route', 'Attempted', 'Succeeded', 'Success Percentage'], dtype='object')

**更改日期格式**  
將日期格式從 mm/dd/yyyy 調整成 yyyy-mm-dd，可用`pd.to_datetime`轉換，從時間排序的角度來說，後者比較便於排序

In [8]:
data1.Date = pd.to_datetime(data1.Date) # covert into datetime type
data2.Date = pd.to_datetime(data2.Date)
data1.head()

,Date,Route,Attempted,Succeeded,Success Percentage
0,2015-11-27,Disappointment Cleaver,2,0,0.0
1,2015-11-21,Disappointment Cleaver,3,0,0.0
2,2015-10-15,Disappointment Cleaver,2,0,0.0
3,2015-10-13,Little Tahoma,8,0,0.0
4,2015-10-09,Disappointment Cleaver,2,0,0.0


**排序日期**  
使用`df.sort_values("columns name",ascending = True)`排序，預設排序是遞增，遞減`ascending = False`

In [9]:
data1 = data1.sort_values("Date")
data2 = data2.sort_values("Date")
data1.head()

,Date,Route,Attempted,Succeeded,Success Percentage
4076,2014-01-04,Gibralter Ledges,3,2,0.666667
4075,2014-01-05,Disappointment Cleaver,2,0,0.000000
4073,2014-01-06,Disappointment Cleaver,8,0,0.000000
4074,2014-01-06,Disappointment Cleaver,8,0,0.000000
4072,2014-01-16,Little Tahoma,2,0,0.000000


**更改小數位數**  
可以用`df.round()`調整小數後幾位，預設是小數後一位。對整個dataframe使用`.round`的話，只有dtype是float的資料才會有更動。

In [10]:
data1 = data1.round(2) # float to two decimal points
data2 = data2.round(2)
data1.head()

,Date,Route,Attempted,Succeeded,Success Percentage
4076,2014-01-04,Gibralter Ledges,3,2,0.67
4075,2014-01-05,Disappointment Cleaver,2,0,0.00
4073,2014-01-06,Disappointment Cleaver,8,0,0.00
4074,2014-01-06,Disappointment Cleaver,8,0,0.00
4072,2014-01-16,Little Tahoma,2,0,0.00


---

## 資料合併<a id =7 ><a>

### Merge
`pd.merge`在合併資料，大部分只用參數`on`,`how`，不過`on`還可分成`left_on`,`right_on`,`left_index`,`right_index`，用法可參考下表：
<img src="./圖片1.jpg" width=50%>

`pd.merge`更詳細的參數介紹，可以參考[Pandans官方文件](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html)

參數`how`有四種合併類型：
* left
* right
* inner
* outer 

以上四種跟SQL的 `left join`, `right join`, `inner join`, `full out join` 相似。
這些類型合併後的差異，可以參考SQL的join關係圖：
<img src = "https://i.stack.imgur.com/VQ5XP.png">   
圖片來源：https://stackoverflow.com/questions/5706437/whats-the-difference-between-inner-join-left-join-right-join-and-full-join

其他更詳細的資料合併介紹，可以參考這兩篇文章：  
[Merge, join, and concatenate](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html)(Pandas官方文件)  
[Combining Datasets: Merge and Join](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html#Specifying-Set-Arithmetic-for-Joins)


使用`pd.merge()`合併，data1跟data2有共同的欄位 Date 可以當連結鍵，所以使用`on = "Data"`。合併的目的是想要看到天氣**對登頂的影響**，因此左側的data1(登山統計)是主要資料，有沒有匹配到data2(天氣)都要顯示data1，所以使用`how = "left"`

In [11]:
df_merge = pd.merge(data1,data2,on ="Date",how = "left")
display(data1.head(),data2.head(),df_merge.head())

,Date,Route,Attempted,Succeeded,Success Percentage
4076,2014-01-04,Gibralter Ledges,3,2,0.67
4075,2014-01-05,Disappointment Cleaver,2,0,0.00
4073,2014-01-06,Disappointment Cleaver,8,0,0.00
4074,2014-01-06,Disappointment Cleaver,8,0,0.00
4072,2014-01-16,Little Tahoma,2,0,0.00


,Date,Battery Voltage AVG,Temperature AVG,Relative Humidity AVG,Wind Speed Daily AVG,Wind Direction AVG,Solare Radiation AVG
463,2014-09-23,13.06,32.86,100.00,65.11,254.27,16.59
462,2014-09-24,13.17,29.70,100.00,45.92,278.05,37.87
461,2014-09-25,13.65,26.82,99.85,3.17,278.60,142.70
460,2014-09-26,13.53,28.24,100.00,0.90,259.65,114.40
459,2014-09-27,13.47,33.32,60.20,7.14,116.29,227.60


,Date,Route,Attempted,Succeeded,Success Percentage,Battery Voltage AVG,Temperature AVG,Relative Humidity AVG,Wind Speed Daily AVG,Wind Direction AVG,Solare Radiation AVG
0,2014-01-04,Gibralter Ledges,3,2,0.67,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-01-05,Disappointment Cleaver,2,0,0.00,NaN,NaN,NaN,NaN,NaN,NaN
2,2014-01-06,Disappointment Cleaver,8,0,0.00,NaN,NaN,NaN,NaN,NaN,NaN
3,2014-01-06,Disappointment Cleaver,8,0,0.00,NaN,NaN,NaN,NaN,NaN,NaN
4,2014-01-16,Little Tahoma,2,0,0.00,NaN,NaN,NaN,NaN,NaN,NaN


### Concat

Concat在一般情況資料合併是不考慮欄位之間的關聯性，直接使用索引合併

In [12]:
df_concat = pd.concat([data1,data2], axis = 1)
display(data1.head(),data2.head(),df_concat.head())

,Date,Route,Attempted,Succeeded,Success Percentage
4076,2014-01-04,Gibralter Ledges,3,2,0.67
4075,2014-01-05,Disappointment Cleaver,2,0,0.00
4073,2014-01-06,Disappointment Cleaver,8,0,0.00
4074,2014-01-06,Disappointment Cleaver,8,0,0.00
4072,2014-01-16,Little Tahoma,2,0,0.00


,Date,Battery Voltage AVG,Temperature AVG,Relative Humidity AVG,Wind Speed Daily AVG,Wind Direction AVG,Solare Radiation AVG
463,2014-09-23,13.06,32.86,100.00,65.11,254.27,16.59
462,2014-09-24,13.17,29.70,100.00,45.92,278.05,37.87
461,2014-09-25,13.65,26.82,99.85,3.17,278.60,142.70
460,2014-09-26,13.53,28.24,100.00,0.90,259.65,114.40
459,2014-09-27,13.47,33.32,60.20,7.14,116.29,227.60


,Date,Route,Attempted,Succeeded,Success Percentage,Date,Battery Voltage AVG,Temperature AVG,Relative Humidity AVG,Wind Speed Daily AVG,Wind Direction AVG,Solare Radiation AVG
0,2015-11-27,Disappointment Cleaver,2,0,0.0,2015-12-31,13.84,19.06,21.87,21.98,62.33,84.92
1,2015-11-21,Disappointment Cleaver,3,0,0.0,2015-12-30,13.82,14.63,18.49,3.54,121.51,86.19
2,2015-10-15,Disappointment Cleaver,2,0,0.0,2015-12-29,13.83,6.61,34.07,0.00,130.29,85.10
3,2015-10-13,Little Tahoma,8,0,0.0,2015-12-28,13.71,8.69,70.56,0.00,164.68,86.24
4,2015-10-09,Disappointment Cleaver,2,0,0.0,2015-12-27,13.36,14.14,95.75,0.00,268.48,31.09


如果要跟merge相同效果的合併，就要將連結鍵設定成索引，再用索引當作合併的依據

In [13]:
df_concat = pd.concat([data1.set_index("Date"),data2.set_index("Date")], axis = 1, join_axes = [data1.set_index("Date").index])
display(df_merge.head(),df_concat.head())

,Date,Route,Attempted,Succeeded,Success Percentage,Battery Voltage AVG,Temperature AVG,Relative Humidity AVG,Wind Speed Daily AVG,Wind Direction AVG,Solare Radiation AVG
0,2014-01-04,Gibralter Ledges,3,2,0.67,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-01-05,Disappointment Cleaver,2,0,0.00,NaN,NaN,NaN,NaN,NaN,NaN
2,2014-01-06,Disappointment Cleaver,8,0,0.00,NaN,NaN,NaN,NaN,NaN,NaN
3,2014-01-06,Disappointment Cleaver,8,0,0.00,NaN,NaN,NaN,NaN,NaN,NaN
4,2014-01-16,Little Tahoma,2,0,0.00,NaN,NaN,NaN,NaN,NaN,NaN


,Route,Attempted,Succeeded,Success Percentage,Battery Voltage AVG,Temperature AVG,Relative Humidity AVG,Wind Speed Daily AVG,Wind Direction AVG,Solare Radiation AVG
Date,,,,,,,,,,
2014-01-04,Gibralter Ledges,3,2,0.67,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-05,Disappointment Cleaver,2,0,0.00,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-06,Disappointment Cleaver,8,0,0.00,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-06,Disappointment Cleaver,8,0,0.00,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-16,Little Tahoma,2,0,0.00,NaN,NaN,NaN,NaN,NaN,NaN
